In [3]:
# Level-3 Data Preparation — Design + Deterministic Code Skeleton
#
# Purpose:
#   Deterministically convert Level-2 training examples (utterance + intent)
#   into a Level-3, reasoning-ready record with explicit symbolic facts,
#   activated constraints, and a derived intent possibility space.
#
# Design principles (compiler-style):
# - Deterministic: keyword tables, regexes, and simple syntactic heuristics.
# - Explainable: every fact and constraint is rule-based and auditable.
# - Reproducible: same input -> same output.
# - Non-probabilistic: no LLMs, no statistical inference, no randomness.
#
# Assumptions about input rows: a mapping-like object with keys:
#   - 'utterance' (str)
#   - 'intent' (one of: 'investigate','execute','summarize','ops')
#
# Target output schema (per record):
#   {
#     'utterance': <str>,
#     'gold_intent': <str>,
#     'facts': { ... boolean / categorical facts ... },
#     'active_constraints': [ ... symbolic constraints ... ],
#     'allowed_intents': [ ... ],
#     'suppressed_intents': [ ... ]
#   }
#
# ------------------------------------------------------------
import re
from typing import Dict, List, Mapping, Any, Tuple
# Canonical intent space (do not add or remove intents)
INTENT_SPACE = ['investigate', 'execute', 'summarize', 'ops']
# Keyword tables and patterns used by the deterministic extractor.
QUESTION_WORDS = ['who','what','when','where','why','how']
IMPERATIVE_VERBS = [
    'restart','reboot','delete','remove','kill','terminate','scale','deploy',
    'push','apply','update','upgrade','rollback','start','stop','reconcile',
    'reconfigure','clean','clear','fix','patch'
]
UNCERTAINTY_MARKERS = ['maybe','might','could','possibly','probably','not sure','unclear','uncertain']
TEMPORAL_PATTERNS = [
    # simple regex fragments; used with re.IGNORECASE
    r'\b(now|immediately|asap|right now)\b',
    r'\b(last|past|previous|since)\b',
    r'\b(in the last|for the last|over the last)\b',
    r'\b(\d+\s?(s|m|h|d|minutes?|hours?|days?))\b',
    r"\b(today|yesterday|tomorrow|this morning|this afternoon|this evening)\b"
]
SYSTEM_STATE_KEYWORDS = ['cpu','memory','latency','error','fail','crash','down','status','uptime','spike','throttle','overloaded','slow','leak']
ACTION_KEYWORDS = IMPERATIVE_VERBS + ['check','show','diagnose','investigate','explain','report','summarize']
# Precompile regexes
TEMPORAL_REGEXES = [re.compile(pat, re.IGNORECASE) for pat in TEMPORAL_PATTERNS]
QUESTION_REGEX = re.compile(r"\?\s*$")  # trailing question mark heuristic
# ------------------------------------------------------------

def extract_facts(utterance: str) -> Dict[str, Any]:
    """Deterministically extract observable facts from an utterance.

    Rules and heuristics used (explicit):

    - is_question: True if the utterance ends with a question mark OR begins with a question word.

    - has_imperative: True if the first token is an imperative verb from a canonical table OR the
      utterance begins with an imperative verb (case-insensitive).

    - has_uncertainty_marker: True if any token/phrase from the uncertainty markers table appears.

    - has_temporal_reference: True if any simple temporal regex matches.

    - mentions_action: True if action keywords appear anywhere (deterministic keyword match).

    - mentions_system_state: True if system-state keywords appear anywhere.

    The function returns a facts dict composed of boolean flags and small categorical fields.

    Deterministic: only exact keyword matches, simple regexes, and token-position heuristics.

    """
    if utterance is None:
        utterance = ''
    u = utterance.strip()
    u_lower = u.lower()
    tokens = re.findall(r"\w+", u_lower)
    # is_question
    starts_with_qword = len(tokens) > 0 and tokens[0] in QUESTION_WORDS
    ends_with_qmark = bool(QUESTION_REGEX.search(u))
    is_question = starts_with_qword or ends_with_qmark
    # has_imperative: check first token against imperative verbs table
    has_imperative = False
    if len(tokens) > 0 and tokens[0] in IMPERATIVE_VERBS:
        has_imperative = True
    # Also consider utterances that begin with a known action verb phrase (two-word checks)
    if not has_imperative:
        first_two = ' '.join(tokens[:2])
        if any(first_two.startswith(v) for v in IMPERATIVE_VERBS):
            has_imperative = True
    # uncertainty markers
    has_uncertainty_marker = any(marker in u_lower for marker in UNCERTAINTY_MARKERS)
    # temporal references
    has_temporal_reference = any(rx.search(u) for rx in TEMPORAL_REGEXES)
    # mentions_action/system_state
    mentions_action = any(word in u_lower for word in ACTION_KEYWORDS)
    mentions_system_state = any(word in u_lower for word in SYSTEM_STATE_KEYWORDS)
    # Provide minimal explainable categorical facts if helpful
    question_word = tokens[0] if starts_with_qword else None
    imperative_verb = tokens[0] if has_imperative else None
    facts = {
        'is_question': bool(is_question),
        'question_word': question_word,
        'has_imperative': bool(has_imperative),
        'imperative_verb': imperative_verb,
        'has_uncertainty_marker': bool(has_uncertainty_marker),
        'has_temporal_reference': bool(has_temporal_reference),
        'mentions_action': bool(mentions_action),
        'mentions_system_state': bool(mentions_system_state),
    }
    return facts


def activate_constraints(facts: Mapping[str, Any]) -> List[str]:
    """Deterministically activate symbolic constraints based on extracted facts.

    Constraints are explicit, testable, and explainable. Example mapping rules:

    - is_question -> 'block_execute' (questions should not trigger direct executes)

    - has_uncertainty_marker -> 'prefer_investigate' (bias towards investigation)

    - has_imperative -> 'allow_execute' (imperative language signals operational action)

    - mentions_system_state and is_question -> 'require_diagnostic_investigate'

    - has_temporal_reference with imperative -> 'require_immediacy_flag' (informational only)

    Each rule is a boolean test over facts; output is a list of symbolic constraint strings.

    """
    constraints: List[str] = []
    if facts.get('is_question'):
        constraints.append('block_execute')
    if facts.get('has_uncertainty_marker'):
        constraints.append('prefer_investigate')
    if facts.get('has_imperative'):
        constraints.append('allow_execute')
    if facts.get('mentions_system_state') and facts.get('is_question'):
        constraints.append('require_diagnostic_investigate')
    if facts.get('has_temporal_reference') and facts.get('has_imperative'):
        constraints.append('require_immediacy_flag')
    # Ensure constraints are unique and ordered deterministically
    unique_constraints = []
    for c in constraints:
        if c not in unique_constraints:
            unique_constraints.append(c)
    return unique_constraints


def derive_intent_space(constraints: List[str]) -> Tuple[List[str], List[str]]:
    """Deterministically derive allowed and suppressed intents from active constraints.

    Principles (explicit):

    - Start with the full canonical intent set as allowed.

    - Apply constraints in a fixed order to remove (suppress) or re-allow intents.

    - The function returns (allowed_intents, suppressed_intents) as lists.

    Rules (examples):

    - 'block_execute' -> remove 'execute' from allowed (suppress execute)

    - 'prefer_investigate' -> ensure 'investigate' is in allowed (no-op if already)

    - 'allow_execute' -> ensure 'execute' is in allowed (if conflicting, allow_execute wins only if block_execute absent)

    - 'require_diagnostic_investigate' -> add 'investigate' and suppress 'execute'

    Note: This function is entirely symbolic and deterministic.

    """
    allowed = list(INTENT_SPACE)  # deterministic copy
    suppressed: List[str] = []
    # Apply rules in a deterministic sequence
    if 'block_execute' in constraints:
        if 'execute' in allowed:
            allowed.remove('execute')
            suppressed.append('execute')
    if 'require_diagnostic_investigate' in constraints:
        # ensure investigate present
        if 'investigate' not in allowed:
            allowed.append('investigate')
        if 'execute' in allowed:
            allowed.remove('execute')
            if 'execute' not in suppressed:
                suppressed.append('execute')
    if 'prefer_investigate' in constraints:
        if 'investigate' not in allowed:
            allowed.append('investigate')
    # allow_execute only re-allows execute when block_execute is absent
    if 'allow_execute' in constraints and 'block_execute' not in constraints:
        if 'execute' not in allowed:
            allowed.append('execute')
        if 'execute' in suppressed:
            suppressed.remove('execute')
    # Final deterministic ordering
    allowed_sorted = [i for i in INTENT_SPACE if i in allowed]
    suppressed_sorted = [i for i in INTENT_SPACE if i in suppressed]
    return allowed_sorted, suppressed_sorted


def compile_l3_record(row: Mapping[str, Any]) -> Dict[str, Any]:
    """Compile a single Level-3 record from an input row.

    Expected input keys: 'utterance' (str), 'intent' (gold label).

    Returns a dict following the Level-3 schema described above.

    This function performs only deterministic transformations and does NOT mutate
    the original gold intent label.

    """
    utterance = (row.get('utterance') or '').strip()
    gold_intent = row.get('intent')
    facts = extract_facts(utterance)
    active_constraints = activate_constraints(facts)
    allowed_intents, suppressed_intents = derive_intent_space(active_constraints)
    record = {
        'utterance': utterance,
        'gold_intent': gold_intent,
        'facts': facts,
        'active_constraints': active_constraints,
        'allowed_intents': allowed_intents,
        'suppressed_intents': suppressed_intents,
    }
    return record

# End of cell: deterministic, auditable, Python-only skeleton ready for use in later cells.
# Note: This cell contains no execution statements, prints, or randomness. It is safe
# to import into downstream notebooks for deterministic Level-3 dataset compilation.

In [4]:
# Cell 2 — Compile Level-3 dataset and save to level3/data (runnable)
import os
import pandas as pd
from typing import Any, Mapping

# Robust repo root finder (replicated to be self-contained in this cell)
def find_repo_root(start_dir=None):
    d = start_dir or os.getcwd()
    while True:
        if os.path.exists(os.path.join(d, 'requirements.txt')) or os.path.exists(os.path.join(d, '.git')):
            return d
        parent = os.path.dirname(d)
        if parent == d:
            return os.getcwd()
        d = parent

repo_root = find_repo_root()
input_path = os.path.join(repo_root, 'data', 'intents_base.csv')

if not os.path.exists(input_path):
    raise FileNotFoundError(f"Expected input CSV not found at {input_path}")

# Load input CSV (expects columns: 'utterance', 'intent')
df = pd.read_csv(input_path)

# Validate required columns
if 'utterance' not in df.columns or 'intent' not in df.columns:
    raise ValueError("Input CSV must contain 'utterance' and 'intent' columns")

# Compile Level-3 records deterministically using compile_l3_record from Cell 1
records = []
for _, row in df.iterrows():
    # Use a plain mapping to avoid pandas-specific behavior
    mapping: Mapping[str, Any] = {'utterance': row['utterance'], 'intent': row['intent']}
    rec = compile_l3_record(mapping)
    records.append(rec)

out_df = pd.DataFrame(records)

# Ensure level3/data exists and save
out_dir = os.path.join(repo_root, 'level3', 'data')
os.makedirs(out_dir, exist_ok=True)
out_path = os.path.join(out_dir, 'level3_intents.csv')
out_df.to_csv(out_path, index=False)

print(f"✓ Wrote {len(out_df)} Level-3 records to: {out_path}")

✓ Wrote 614 Level-3 records to: c:\git\nsai_poc\level3\data\level3_intents.csv


In [7]:
# Cell 3 — Constraint Taxonomy & Governance (declarative, runnable)
# canonical registry of constraints used by Level-3 reasoning
CONSTRAINT_REGISTRY = {
    'block_execute': {
        'type': 'hard',
        'description': 'Prevent direct execution when present',
        'rationale': 'Questions/clarifications should not trigger executes'
    },
    'prefer_investigate': {
        'type': 'soft',
        'description': 'Prefer investigation over execution',
        'rationale': 'Uncertainty markers bias toward investigative actions'
    },
    'allow_execute': {
        'type': 'soft',
        'description': 'Allow execution when imperative language is detected',
        'rationale': 'Imperative phrasing indicates operator intent'
    },
    'require_diagnostic_investigate': {
        'type': 'hard',
        'description': 'Require a diagnostic investigation for system-state questions',
        'rationale': 'System-state inquiries require diagnostics before action'
    },
    'require_immediacy_flag': {
        'type': 'soft',
        'description': 'Mark request as immediate/urgent when temporal + imperative',
        'rationale': 'Communicates urgency for downstream handling'
    }
}

# This registry is declarative, importable, and intentionally computation-free.


In [11]:
# Cell 4 — Dataset Sanity & Logical Consistency Checks
import os
import pandas as pd
import ast
from typing import Any

# load compiled level3 CSV
repo_root = os.getcwd()
for _ in range(6):
    if os.path.exists(os.path.join(repo_root, 'requirements.txt')) or os.path.exists(os.path.join(repo_root, '.git')):
        break
    repo_root = os.path.dirname(repo_root)
level3_path = os.path.join(repo_root, 'level3', 'data', 'level3_intents.csv')
if not os.path.exists(level3_path):
    raise FileNotFoundError(level3_path)

df = pd.read_csv(level3_path)

# safe parser for list-like cells
def _parse_list_cell(x: Any):
    if isinstance(x, (list, tuple)):
        return list(x)
    if pd.isna(x):
        return []
    if isinstance(x, str):
        x = x.strip()
        if x == '':
            return []
        try:
            val = ast.literal_eval(x)
            if isinstance(val, (list, tuple)):
                return list(val)
        except Exception:
            return [x]
    return []

# Normalize expected list-columns
for c in ['allowed_intents','suppressed_intents','active_constraints']:
    if c in df.columns:
        df[c] = df[c].apply(_parse_list_cell)

# checks
if 'gold_intent' not in df.columns:
    raise ValueError('gold_intent column missing')

# conservative normalization for checking only (prefix-based rules)
def _normalize_intent_for_check(intent: Any):
    if not isinstance(intent, str):
        return intent
    k = intent.strip().lower()
    if k in INTENT_SPACE:
        return k
    if k.startswith('execut'):
        return 'execute'
    if k.startswith('summar'):
        return 'summarize'
    if k.startswith('investig'):
        return 'investigate'
    if k.startswith('op'):
        return 'ops'
    return k


def _gold_not_allowed(row):
    gi = _normalize_intent_for_check(row['gold_intent'])
    ai = [a.strip().lower() for a in (row.get('allowed_intents') or [])]
    return gi not in ai

def _gold_in_suppressed(row):
    gi = _normalize_intent_for_check(row['gold_intent'])
    si = [s.strip().lower() for s in (row.get('suppressed_intents') or [])]
    return gi in si

def _empty_allowed(row):
    ai = row.get('allowed_intents') or []
    return len(ai) == 0

viol_gold_not_allowed = df.apply(_gold_not_allowed, axis=1)
viol_gold_in_suppressed = df.apply(_gold_in_suppressed, axis=1)
viol_empty_allowed = df.apply(_empty_allowed, axis=1)

count_not_allowed = int(viol_gold_not_allowed.sum())
count_in_suppressed = int(viol_gold_in_suppressed.sum())
count_empty_allowed = int(viol_empty_allowed.sum())

total = len(df)
print('Dataset Sanity Checks')
print('Total records:', total)
print(f"gold_intent not in allowed_intents: {count_not_allowed}")
print(f"gold_intent in suppressed_intents: {count_in_suppressed}")
print(f"allowed_intents empty: {count_empty_allowed}")

# small sample of violating rows (if any)
viol_any = df[viol_gold_not_allowed | viol_gold_in_suppressed | viol_empty_allowed]
if len(viol_any) > 0:
    print('\nSample violating rows (up to 10):')
    cols = ['utterance','gold_intent','allowed_intents','suppressed_intents','active_constraints']
    cols = [c for c in cols if c in df.columns]
    sample = viol_any[cols].head(10).copy()
    for c in ['allowed_intents','suppressed_intents','active_constraints']:
        if c in sample.columns:
            sample[c] = sample[c].apply(lambda v: repr(v))
    print(sample.to_string(index=False))

# fail-fast threshold (configurable); if exceeded, report WARNING but do not raise
threshold = max(5, int(0.01 * total))
if len(viol_any) > threshold:
    print(f"WARNING: Too many logical violations: {len(viol_any)} > {threshold} — proceeding (review required)")

# expose counts and pass/fail flag
SANITY_COUNTS = {
    'total': int(total),
    'gold_not_allowed': count_not_allowed,
    'gold_in_suppressed': count_in_suppressed,
    'empty_allowed': count_empty_allowed,
}
SANITY_OK = len(viol_any) == 0


Dataset Sanity Checks
Total records: 614
gold_intent not in allowed_intents: 169
gold_intent in suppressed_intents: 0
allowed_intents empty: 0

Sample violating rows (up to 10):
                    utterance  gold_intent                                allowed_intents suppressed_intents active_constraints
            hello how are you out_of_scope ['investigate', 'execute', 'summarize', 'ops']                 []                 []
what is the capital of france out_of_scope            ['investigate', 'summarize', 'ops']        ['execute']  ['block_execute']
               tell me a joke out_of_scope ['investigate', 'execute', 'summarize', 'ops']                 []                 []
        who won the world cup out_of_scope            ['investigate', 'summarize', 'ops']        ['execute']  ['block_execute']
   write a poem about servers out_of_scope ['investigate', 'execute', 'summarize', 'ops']                 []                 []
     how is the weather today out_of_scope            

In [ ]:
# Cell 5 — Coverage & Distribution Diagnostics
import os
import pandas as pd
import ast
from collections import Counter
from typing import Any

# load compiled level3 CSV
repo_root = os.getcwd()
for _ in range(6):
    if os.path.exists(os.path.join(repo_root, 'requirements.txt')) or os.path.exists(os.path.join(repo_root, '.git')):
        break
    repo_root = os.path.dirname(repo_root)
level3_path = os.path.join(repo_root, 'level3', 'data', 'level3_intents.csv')
if not os.path.exists(level3_path):
    raise FileNotFoundError(level3_path)

df = pd.read_csv(level3_path)

# safe parser for list-like cells (reuse same logic as sanity cell)
def _parse_list_cell(x: Any):
    if isinstance(x, (list, tuple)):
        return list(x)
    if pd.isna(x):
        return []
    if isinstance(x, str):
        x = x.strip()
        if x == '':
            return []
        try:
            val = ast.literal_eval(x)
            if isinstance(val, (list, tuple)):
                return list(val)
        except Exception:
            return [x]
    return []

for c in ['active_constraints','suppressed_intents','allowed_intents']:
    if c in df.columns:
        df[c] = df[c].apply(_parse_list_cell)

# frequency of each constraint
all_constraints = []
for row in df['active_constraints']:
    all_constraints.extend(row)
constraint_counts = Counter(all_constraints)

print('Constraint frequencies (constraint: count)')
for k,v in sorted(constraint_counts.items()):
    print(f'  {k}: {v}')

# how often each intent is suppressed
suppressed_list = []
if 'suppressed_intents' in df.columns:
    for s in df['suppressed_intents']:
        suppressed_list.extend(s)
suppressed_counts = Counter(suppressed_list)
print('\nSuppressed intent frequencies (intent: count)')
for k,v in sorted(suppressed_counts.items()):
    print(f'  {k}: {v}')

# common constraint combinations (deterministic ordering)
combo_counts = Counter()
for row in df['active_constraints']:
    combo = tuple(row)  # keep order as-is (deterministic)
    combo_counts[combo] += 1

print('\nTop constraint combinations (combo -> count)')
for combo, cnt in combo_counts.most_common(10):
    print(f'  {list(combo)} -> {cnt}')

# expose summaries
CONSTRAINT_FREQUENCIES = dict(sorted(constraint_counts.items()))
SUPPRESSED_INTENT_FREQUENCIES = dict(sorted(suppressed_counts.items()))
COMMON_CONSTRAINT_COMBOS = [(list(c), n) for c,n in combo_counts.most_common(10)]


Constraint frequencies (constraint: count)
   : 17
  ': 522
  ,: 17
  [: 614
  ]: 614
  _: 278
  a: 99
  b: 179
  c: 440
  d: 17
  e: 800
  g: 34
  i: 85
  k: 179
  l: 309
  n: 34
  o: 261
  q: 17
  r: 34
  s: 34
  t: 295
  u: 261
  v: 17
  w: 65
  x: 244

Suppressed intent frequencies (intent: count)
  ': 358
  [: 614
  ]: 614
  c: 179
  e: 537
  t: 179
  u: 179
  x: 179

Top constraint combinations (combo -> count)
  ['[', ']'] -> 370
  ['[', "'", 'b', 'l', 'o', 'c', 'k', '_', 'e', 'x', 'e', 'c', 'u', 't', 'e', "'", ']'] -> 162
  ['[', "'", 'a', 'l', 'l', 'o', 'w', '_', 'e', 'x', 'e', 'c', 'u', 't', 'e', "'", ']'] -> 65
  ['[', "'", 'b', 'l', 'o', 'c', 'k', '_', 'e', 'x', 'e', 'c', 'u', 't', 'e', "'", ',', ' ', "'", 'r', 'e', 'q', 'u', 'i', 'r', 'e', '_', 'd', 'i', 'a', 'g', 'n', 'o', 's', 't', 'i', 'c', '_', 'i', 'n', 'v', 'e', 's', 't', 'i', 'g', 'a', 't', 'e', "'", ']'] -> 17


In [ ]:
# Cell 6 — Level-3 Readiness Summary (final, lightweight)
import os
import pandas as pd
import ast

# locate file
repo_root = os.getcwd()
# try to find repo root similarly to previous cells
for _ in range(6):
    if os.path.exists(os.path.join(repo_root, 'requirements.txt')) or os.path.exists(os.path.join(repo_root, '.git')):
        break
    repo_root = os.path.dirname(repo_root)
level3_path = os.path.join(repo_root, 'level3', 'data', 'level3_intents.csv')
if not os.path.exists(level3_path):
    raise FileNotFoundError(f"Level-3 file not found at {level3_path}")

df = pd.read_csv(level3_path)
# parse list/dict-like columns if saved as strings
for c in ['active_constraints','allowed_intents','suppressed_intents']:
    if c in df.columns:
        if df[c].dtype == object:
            def _parse(x):
                if pd.isna(x):
                    return []
                if isinstance(x, (list, tuple)):
                    return x
                try:
                    return ast.literal_eval(x)
                except Exception:
                    return []
            df[c] = df[c].apply(_parse)

# helper: detect hard constraints using registry if available
try:
    from level3.level3_data_prep import CONSTRAINT_REGISTRY  # avoid circular import; falls back
except Exception:
    # try to import from this notebook's namespace if present
    CONSTRAINT_REGISTRY = globals().get('CONSTRAINT_REGISTRY', None)

hard_set = set()
if isinstance(CONSTRAINT_REGISTRY, dict):
    hard_set = {k for k,v in CONSTRAINT_REGISTRY.items() if v.get('type') == 'hard'}

# compute readiness metrics
total = len(df)
rows_with_hard = 0
rows_with_suppressed = 0
for _, r in df.iterrows():
    ac = r.get('active_constraints') or []
    if any((c in hard_set) for c in ac):
        rows_with_hard += 1
    si = r.get('suppressed_intents') or []
    if len(si) > 0:
        rows_with_suppressed += 1

pct_hard = rows_with_hard / total * 100 if total else 0.0
pct_suppressed = rows_with_suppressed / total * 100 if total else 0.0

# recompute logical violations (as defensive check)
import ast as _ast

def _parse_list_cell(x):
    if pd.isna(x):
        return []
    if isinstance(x, (list, tuple)):
        return x
    try:
        return _ast.literal_eval(x)
    except Exception:
        return []

if 'allowed_intents' in df.columns and 'suppressed_intents' in df.columns and 'gold_intent' in df.columns:
    allowed = df['allowed_intents'].apply(_parse_list_cell)
    suppressed = df['suppressed_intents'].apply(_parse_list_cell)
    gold = df['gold_intent']
    violations = ((~allowed.apply(lambda a: gold.iloc[0] in a)) if False else None)  # placeholder
    # compute rowwise violation counts
    row_viol = []
    for i in range(len(df)):
        gi = gold.iloc[i]
        ai = allowed.iloc[i]
        si = suppressed.iloc[i]
        v = 0
        if gi not in ai:
            v += 1
        if gi in si:
            v += 1
        if len(ai) == 0:
            v += 1
        row_viol.append(v)
    total_violations = sum(1 for v in row_viol if v > 0)
else:
    total_violations = 0

# verdict
verdict = 'LEVEL-3 DATASET READY' if total_violations == 0 else 'LEVEL-3 DATASET BLOCKED'

# machine-readable summary
summary = {
    'total_records': total,
    'percent_with_hard_constraints': round(pct_hard, 2),
    'percent_with_suppressed_intents': round(pct_suppressed, 2),
    'logical_violations_count': int(total_violations),
    'verdict': verdict
}

# human-readable summary
print('LEVEL-3 Readiness Summary')
print('Total records:', total)
print(f"% with hard constraints: {summary['percent_with_hard_constraints']}%")
print(f"% with suppressed intents: {summary['percent_with_suppressed_intents']}%")
print('Logical violations (rowwise):', summary['logical_violations_count'])
print('\nVERDICT:', summary['verdict'])

# expose summary variable for downstream cells
LEVEL3_READINESS_SUMMARY = summary
